In [19]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from models import *   
import load_data
import nan_imputation
import helpers
from helpers import find_repo_root

# Step 0 : load the data from Optogenetics

In [26]:
import importlib
importlib.reload(load_data)
importlib.reload(helpers)

<module 'helpers' from '/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/helpers.py'>

In [22]:
repo_root = find_repo_root()
repo_root

data_path = os.path.join(repo_root, 'Data/Optogenetics')

In [27]:
worms = load_data.load_optogenetics(data_path)

Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Optogenetics/ATR-/speeds_and_coordinates_13piworm14_5_updated.csv
File loaded: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Optogenetics/ATR-/speeds_and_coordinates_13piworm14_5_updated.csv
   Frame     Speed           X           Y  Changed Pixels  Light_Pulse
0      1  0.159865  336.837398  285.260163               3            0
1      2  1.391898  336.822581  285.338710              23            0
2      3  0.045343  337.015038  286.007519               2            0
3      4  0.000000  336.992366  286.007634               0            0
4      5  0.144837  336.992366  286.007634               2            0
Loading file: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Optogenetics/ATR-/speeds_and_coordinates_27piworm16_4_updated.csv
File loaded: /Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Optogenetics

In [31]:
# just a check print on worm 17 (companyDrug)
worm_name = 'worm_17'  # Change this to the name of the worm you want to print
print(f"Worm: {worm_name}")
worm_data = worms[worm_name]
df_17 = pd.DataFrame(worm_data.T, columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Light_Pulse', 'Category'])
df_17

Worm: worm_17


,Frame,Speed,X,Y,Changed Pixels,Light_Pulse,Category
0,1.0,0.655716,636.087591,269.890511,7.0,0.0,0.0
1,2.0,0.460757,636.141791,269.567164,4.0,0.0,0.0
2,3.0,0.358394,636.044776,269.776119,5.0,0.0,0.0
3,4.0,0.472356,635.916031,269.900763,6.0,0.0,0.0
4,5.0,0.833469,635.689922,269.968992,10.0,0.0,0.0
...,...,...,...,...,...,...,...
118784,10795.0,0.999262,685.928205,110.482051,13.0,0.0,0.0
118785,10796.0,0.246228,685.452926,110.636132,7.0,0.0,0.0
118786,10797.0,0.618560,685.335897,110.674359,14.0,0.0,0.0
118787,10798.0,0.370867,685.033333,110.738462,20.0,0.0,0.0


In [32]:
helpers.print_fdict_summary(worms)


Summary of Loaded Worm Data:
Worm: worm_1
  Shape: (7, 140387)
----------------------------------------
Worm: worm_2
  Shape: (7, 140387)
----------------------------------------
Worm: worm_3
  Shape: (7, 140387)
----------------------------------------
Worm: worm_4
  Shape: (7, 140387)
----------------------------------------
Worm: worm_5
  Shape: (7, 140387)
----------------------------------------
Worm: worm_6
  Shape: (7, 140387)
----------------------------------------
Worm: worm_7
  Shape: (7, 118789)
----------------------------------------
Worm: worm_8
  Shape: (7, 140387)
----------------------------------------
Worm: worm_9
  Shape: (7, 129588)
----------------------------------------
Worm: worm_10
  Shape: (7, 140387)
----------------------------------------
Worm: worm_11
  Shape: (7, 140387)
----------------------------------------
Worm: worm_12
  Shape: (7, 140387)
----------------------------------------
Worm: worm_13
  Shape: (7, 140387)
--------------------------------

# Step 1 : NaN Imputation

In [33]:
for name, lifespan_array in worms.items(): 
    print(f"Processing {name}")
    lifespan_arrayxy = lifespan_array[2:4,:]  # Extract columns for X and Y
    missing_sequences = nan_imputation.count_successive_missing(lifespan_arrayxy)
    for start, end, length in missing_sequences:
        print(f"  Missing sequence starts at column {start}, ends at column {end - 1}, length: {length}")

Processing worm_1
  Missing sequence starts at column 1321, ends at column 1324, length: 4
  Missing sequence starts at column 2148, ends at column 2152, length: 5
  Missing sequence starts at column 7021, ends at column 7022, length: 2
  Missing sequence starts at column 7355, ends at column 7359, length: 5
  Missing sequence starts at column 7429, ends at column 7445, length: 17
  Missing sequence starts at column 7591, ends at column 7592, length: 2
  Missing sequence starts at column 7595, ends at column 7607, length: 13
  Missing sequence starts at column 27693, ends at column 27707, length: 15
  Missing sequence starts at column 28437, ends at column 28543, length: 107
  Missing sequence starts at column 55483, ends at column 55500, length: 18
  Missing sequence starts at column 60022, ends at column 60023, length: 2
  Missing sequence starts at column 62623, ends at column 62649, length: 27
  Missing sequence starts at column 62811, ends at column 62833, length: 23
  Missing seq

In [34]:
# Rows to check for missing values (2:4 in zero-based indexing)
rows_to_check = slice(2, 4)  # Rows 2 and 3 not row 4

# Apply cut_array to each worm in the dataset
cut_nan_dict = {name: nan_imputation.cut_array(array, rows_to_check) for name, array in worms.items()}

In [35]:
# Print the shape of the filtered arrays
for name, item in cut_nan_dict.items():
    print(f'{name} : {item.shape}')

worm_1 : (7, 138713)
worm_2 : (7, 129380)
worm_3 : (7, 131731)
worm_4 : (7, 136140)
worm_5 : (7, 133752)
worm_6 : (7, 138863)
worm_7 : (7, 115837)
worm_8 : (7, 117069)
worm_9 : (7, 115891)
worm_10 : (7, 111211)
worm_11 : (7, 136025)
worm_12 : (7, 126192)
worm_13 : (7, 133453)
worm_14 : (7, 138230)
worm_15 : (7, 115628)
worm_16 : (7, 76833)
worm_17 : (7, 112990)
worm_18 : (7, 126553)
worm_19 : (7, 117506)
worm_20 : (7, 125588)
worm_21 : (7, 79940)
worm_22 : (7, 137703)
worm_23 : (7, 79982)
worm_24 : (7, 103143)
worm_25 : (7, 132173)
worm_26 : (7, 132924)
worm_27 : (7, 132991)
worm_28 : (7, 120643)
worm_29 : (7, 133625)
worm_30 : (7, 127150)
worm_31 : (7, 113460)
worm_32 : (7, 77023)
worm_33 : (7, 137875)
worm_34 : (7, 138736)
worm_35 : (7, 127897)
worm_36 : (7, 72495)
worm_37 : (7, 132513)
worm_38 : (7, 79758)
worm_39 : (7, 133560)
worm_40 : (7, 137566)
worm_41 : (7, 114925)
worm_42 : (7, 80402)


In [37]:
# just a check print --> If we check we do have the number of frames decreased (because NaNs where removed) --> example with worm_17
worm_name = 'worm_17'  # Change this to the name of the worm you want to print
print(f"Worm: {worm_name}")
worm_data = cut_nan_dict[worm_name]
df = pd.DataFrame(worm_data.T, columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Light_Pulse' ,'Category'])

# Check for NaN values in the DataFrame
if df.isna().sum().sum() == 0:
    print(f"Worm {worm_name} has no NaN values after NaN imputation.")
else:
    print(f"Worm {worm_name} still contains NaN values.")

df

#And we see that the total number of frames is decreased 

Worm: worm_17
Worm worm_17 has no NaN values after NaN imputation.


,Frame,Speed,X,Y,Changed Pixels,Light_Pulse,Category
0,1.0,0.655716,636.087591,269.890511,7.0,0.0,0.0
1,2.0,0.460757,636.141791,269.567164,4.0,0.0,0.0
2,3.0,0.358394,636.044776,269.776119,5.0,0.0,0.0
3,4.0,0.472356,635.916031,269.900763,6.0,0.0,0.0
4,5.0,0.833469,635.689922,269.968992,10.0,0.0,0.0
...,...,...,...,...,...,...,...
112985,10795.0,0.999262,685.928205,110.482051,13.0,0.0,0.0
112986,10796.0,0.246228,685.452926,110.636132,7.0,0.0,0.0
112987,10797.0,0.618560,685.335897,110.674359,14.0,0.0,0.0
112988,10798.0,0.370867,685.033333,110.738462,20.0,0.0,0.0


# Step 2 : Figure out when do the worms die

# Step 5 : plitting the data

In [38]:
train_worms, test_worms = load_data.split_worms(worms, test_size=0.2)

print(f"Training Worms: {list(train_worms.keys())}")
print(f"Testing Worms: {list(test_worms.keys())}")

Split 42 worms into 33 training and 9 testing worms.
Training Worms: ['worm_10', 'worm_4', 'worm_11', 'worm_27', 'worm_38', 'worm_19', 'worm_40', 'worm_26', 'worm_5', 'worm_34', 'worm_23', 'worm_12', 'worm_30', 'worm_22', 'worm_13', 'worm_25', 'worm_36', 'worm_37', 'worm_33', 'worm_14', 'worm_24', 'worm_21', 'worm_29', 'worm_39', 'worm_31', 'worm_20', 'worm_17', 'worm_42', 'worm_35', 'worm_32', 'worm_1', 'worm_3', 'worm_28']
Testing Worms: ['worm_6', 'worm_7', 'worm_9', 'worm_15', 'worm_16', 'worm_18', 'worm_2', 'worm_8', 'worm_41']


For optogenetics dataset:
Atr- should not react to light
Atr+ should react to light 

Can we predict the response to the light but without the light